In [1]:
import pandas as pd
import os

In [6]:
background_df = pd.read_csv("./Resources/firearm-bc.csv")
background_df


,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074,0.0,15.0,...,0.0,0.0,0.0,28.0,25.0,3.0,0.0,1.0,0.0,58549
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208,0.0,2.0,...,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,7400
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239,0.0,7.0,...,2.0,0.0,0.0,12.0,6.0,2.0,0.0,0.0,0.0,43789
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427,14.0,8.0,...,0.0,0.0,0.0,7.0,8.0,1.0,0.0,1.0,0.0,18508
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0,0.0,0.0,...,82.0,0.0,0.0,7708.0,3463.0,647.0,15.0,19.0,18.0,138358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,1998-11,Virginia,0.0,NaN,14.0,2.0,NaN,8,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
15726,1998-11,Washington,1.0,NaN,65.0,286.0,NaN,8,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361
15727,1998-11,West Virginia,3.0,NaN,149.0,251.0,NaN,5,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,408
15728,1998-11,Wisconsin,0.0,NaN,25.0,214.0,NaN,2,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241


In [10]:
#reducing the DF
short_df = background_df[["month", "state","permit","permit_recheck","handgun","long_gun", "other", "multiple"]].copy()

short_df

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0
...,...,...,...,...,...,...,...,...
15725,1998-11,Virginia,0.0,NaN,14.0,2.0,NaN,8
15726,1998-11,Washington,1.0,NaN,65.0,286.0,NaN,8
15727,1998-11,West Virginia,3.0,NaN,149.0,251.0,NaN,5
15728,1998-11,Wisconsin,0.0,NaN,25.0,214.0,NaN,2


In [11]:
#drop row with NAN
reduced_df = short_df.dropna()
reduced_df

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0
...,...,...,...,...,...,...,...,...
4340,2016-02,Virginia,784.0,0.0,30085.0,15948.0,1133.0,0
4341,2016-02,Washington,15736.0,0.0,20583.0,11991.0,1832.0,863
4342,2016-02,West Virginia,3527.0,0.0,10746.0,7436.0,357.0,757
4343,2016-02,Wisconsin,9420.0,0.0,19465.0,12431.0,821.0,62


In [23]:

month_df=reduced_df['month'].str.split('-', expand=True)
year_df= month_df.rename(columns={0: "Year"})
year_df

,Year,1
0,2022,08
1,2022,08
2,2022,08
3,2022,08
4,2022,08
...,...,...
4340,2016,02
4341,2016,02
4342,2016,02
4343,2016,02


In [29]:
y_df= year_df[["Year"]].copy()
y_df

,Year
0,2022
1,2022
2,2022
3,2022
4,2022
...,...
4340,2016
4341,2016
4342,2016
4343,2016


In [38]:
reduced_df.join(y_df)

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,year,month_split,Year
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074,2022,"[2022, 08]",2022
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208,2022,"[2022, 08]",2022
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239,2022,"[2022, 08]",2022
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427,2022,"[2022, 08]",2022
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0,2022,"[2022, 08]",2022
...,...,...,...,...,...,...,...,...,...,...,...
4340,2016-02,Virginia,784.0,0.0,30085.0,15948.0,1133.0,0,2016,"[2016, 02]",2016
4341,2016-02,Washington,15736.0,0.0,20583.0,11991.0,1832.0,863,2016,"[2016, 02]",2016
4342,2016-02,West Virginia,3527.0,0.0,10746.0,7436.0,357.0,757,2016,"[2016, 02]",2016
4343,2016-02,Wisconsin,9420.0,0.0,19465.0,12431.0,821.0,62,2016,"[2016, 02]",2016
